In [1]:
import pypianoroll as ppr 
import numpy as np
import os
from random import random
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras.layers import Dense, LSTM, Flatten, Input, Bidirectional, TimeDistributed, Activation, Concatenate, Embedding, MaxPooling1D, CategoryEncoding, Conv1D, Dropout
from tensorflow.keras.utils import pad_sequences, timeseries_dataset_from_array, split_dataset, to_categorical
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam

import calendar
import time
import shutil
from sklearn.model_selection import train_test_split



In [53]:
INSTRUMENTS = 1
PITCHES = 129

def resample(empties_boolean, prune_percent=0.7):
    resampled_inclusion_beats = [ ]
    
    for isempty in empties_boolean:
        include = True
        if isempty :
            if random() < prune_percent:  #70% pruning of empty beats
                include = False
    
        resampled_inclusion_beats += [include]

    return resampled_inclusion_beats


def store_dataset_as_batches(dataset, dataset_name, dataset_type='train'):

    for batch_id, batch in enumerate(dataset):
        inputs, outputs = batch 
        try:
            np.save(f'lpd_5_batched/{dataset_type}_inputs/{dataset_name}_{batch_id}.npy', inputs)
            np.save(f'lpd_5_batched/{dataset_type}_outputs/{dataset_name}_{batch_id}.npy', outputs)
        except Exception as E:
            print(E)

def make_dataset(dir, samples, chunkid, trackid, resolution, batch_size, prune_rest_note_percent, encoder_decoder, input_sequence_len, output_sequence_len):
    
    track = ppr.load(os.path.join(dir, samples[chunkid+trackid])).binarize().set_resolution(resolution).stack()
            
    if INSTRUMENTS == 1:
        track = track[1:2] # take only guitar for one instrument


    # Move axis of tracks
    track = np.moveaxis(track, (0, 1, 2), (1, 0, 2)) #(time_steps, 5, 128)

    # Concatenate extra dimension at 0 for empty   
    track = np.concatenate([np.zeros(track.shape[:-1] + (1,)), track], axis=-1)
    track[np.any(track, axis=-1)==False, 0] = 1 


    # Argmax results, one pitch at a time step for an instrument, ignores chords
    track = track.argmax(axis=-1)

    #print("Shape before resampling : ", d.shape)
    
    # Resample from empty beats
    empty_beats = (np.sum(track, axis=1) == 0)
    inclusion_beats = resample(empty_beats, prune_percent=prune_rest_note_percent)
    
    track= track[inclusion_beats]

    #print("Shape after resampling : ", d.shape)
    
    try:
        if encoder_decoder:
            input_track = track[:-output_sequence_len]
            output_track = track[input_sequence_len:]
        else:
            input_track = track[:-1]
            output_track = track[1:]
        

        input_dataset = timeseries_dataset_from_array(input_track, None, sequence_length=input_sequence_len, sequence_stride=1, batch_size=batch_size)
        output_dataset = timeseries_dataset_from_array(output_track, None, sequence_length=input_sequence_len, sequence_stride=1, batch_size=batch_size)
        dataset_len = len(input_dataset)
    
        dataset = zip(input_dataset, output_dataset)
        #else:
        #    dataset = timeseries_dataset_from_array(track, track[input_sequence_len:], sequence_length=input_sequence_len, sequence_stride=1, batch_size=batch_size)
        #    dataset_len = len(dataset)
            
    except Exception as E:
        dataset = None
        pass
    
    return dataset 

def process_dataset_in_chunks(dir, input_sequence_len=2400, output_sequence_len=None, batch_size=64, topn=-1, train_size=0.8, val_size=0.2, resolution=24, prune_rest_note_percent=0.3, encoder_decoder=False):
    samples = os.listdir(dir)[:topn]
    train_samples, test_samples = train_test_split(samples, train_size=train_size, shuffle=True)
    train_samples, val_samples = train_test_split(train_samples, test_size=val_size, shuffle=False)

    chunk_size = len(train_samples)//100 #100th of total sample count
    chunk_size = chunk_size if chunk_size else 1
    
    try:
        shutil.rmtree('lpd_5_batched/train_inputs')
        shutil.rmtree('lpd_5_batched/train_outputs')
    except:
        pass

    try:
        shutil.rmtree('lpd_5_batched/val_inputs')
        shutil.rmtree('lpd_5_batched/val_outputs')
    except:
        pass 
    
    try:
        shutil.rmtree('lpd_5_batched/test_inputs')
        shutil.rmtree('lpd_5_batched/test_outputs')
    except:
        pass

    os.makedirs(f'lpd_5_batched/train_inputs')
    os.makedirs(f'lpd_5_batched/train_outputs')

    os.makedirs(f'lpd_5_batched/val_inputs')
    os.makedirs(f'lpd_5_batched/val_outputs')

    os.makedirs(f'lpd_5_batched/test_inputs')
    os.makedirs(f'lpd_5_batched/test_outputs')


    #gmt = time.gmtime()
    #ts = calendar.timegm(gmt)
    for chunkid in tqdm(range(0, len(train_samples),chunk_size),desc='Preparing Training Dataset...'):
        for trackid in range(chunk_size):
            dataset = make_dataset(dir, train_samples, chunkid, trackid, resolution, batch_size, prune_rest_note_percent, encoder_decoder, input_sequence_len, output_sequence_len)
            if dataset:
                store_dataset_as_batches(dataset, dataset_name=f'Pr{os.path.basename(dir)}-Ch{chunkid}-Tr{trackid}', dataset_type='train')
    
    chunk_size = len(val_samples)//100 #100th of total sample count
    chunk_size = chunk_size if chunk_size else 1

    for chunkid in tqdm(range(0, len(val_samples),chunk_size),desc='Preparing Validation Dataset...'):
        for trackid in range(chunk_size):
            dataset = make_dataset(dir, val_samples, chunkid, trackid, resolution, batch_size, prune_rest_note_percent, encoder_decoder, input_sequence_len, output_sequence_len)
            if dataset:
                store_dataset_as_batches(dataset, dataset_name=f'Pr{os.path.basename(dir)}-Ch{chunkid}-Tr{trackid}', dataset_type='val')
            
            

    for trackid in tqdm(range(0, len(test_samples), 1), desc='Preparing Test Dataset...'):
        track = ppr.load(os.path.join(dir, test_samples[trackid])).binarize().set_resolution(resolution).stack()
            
        if INSTRUMENTS == 1:
            track = track[1:2] # take only guitar for one instrument


        # Move axis of tracks
        track = np.moveaxis(track, (0, 1, 2), (1, 0, 2)) #(time_setps, 5, 128)

        input_track = track[:input_sequence_len]
        output_track = track[input_sequence_len:]

        try:
            np.save(f'lpd_5_batched/test_inputs/Pr{os.path.basename(dir)}-Tr{trackid}.npy', input_track)
            np.save(f'lpd_5_batched/test_outputs/Pr{os.path.basename(dir)}-Tr{trackid}.npy', output_track)
        except Exception as E:
            continue


       
    
    return ('lpd_5_batched/train_inputs/', 'lpd_5_batched/train_outputs/'), ('lpd_5_batched/val_inputs/', 'lpd_5_batched/val_outputs/'),  ('lpd_5_batched/test_inputs/', 'lpd_5_batched/test_outputs/')

                        


format_targets = lambda y: tf.unstack(tf.experimental.numpy.moveaxis(y, (0, 1, 2), (1, 2, 0)))[0] if INSTRUMENTS == 1 else tuple(tf.unstack(tf.experimental.numpy.moveaxis(y, (0, 1, 2), (1, 2, 0))))

def load_music_batches(input_dir, output_dir, encoder_decoder=True):

    while 1:
        for inp_batch, output_batch in zip(os.listdir(input_dir), os.listdir(output_dir)):
            
            try:
                inputs, targets = np.load(os.path.join(input_dir, inp_batch)), np.load(os.path.join(output_dir, output_batch))
                
                if encoder_decoder:
                    prompt_inputs = np.concatenate([inputs[:, -2:-1], targets[:, :-1]], axis=1)#none, 2400, 5
                    yield [inputs, prompt_inputs], format_targets(targets)
                else:
                    yield inputs, format_targets(targets)
            except Exception as E:
                continue


import subprocess

def midi_to_wav(midi_path, output_wav_path):
    try:
        # Run Timidity++ command to convert MIDI to WAV
        subprocess.run(["timidity", midi_path, "-Ow", "-o", output_wav_path], check=True)
        print("Conversion completed successfully.")
    except subprocess.CalledProcessError as e:
        print(f"Error: {e}")
        print("Conversion failed.")


def multitrack_to_midi(multitrack, output_path):
    # Check and convert tracks if necessary
    for i, track in enumerate(multitrack.tracks):
        if not isinstance(track, (ppr.BinaryTrack, ppr.StandardTrack)):
            print(f"Converting track {i} to StandardTrack...")
            multitrack.tracks[i] = track.to_pianoroll().to_track()

    # Write the multitrack to a MIDI file
    multitrack.write(output_path)

from random import random, randint


def top_p_sampling(probabilities, p):
    sorted_indices = np.argsort(probabilities)[::-1]  # Sort probabilities in descending order
    cumulative_probs = np.cumsum(probabilities[sorted_indices])  # Compute cumulative probabilities
    if np.any(cumulative_probs <= p):
        selected_indices = sorted_indices[cumulative_probs <= p]  # Select indices where cumulative probability <= p
    else:
        # If none of the cumulative probabilities exceed p, select the maximum probability
        selected_indices = np.array([np.argmax(probabilities)])
    return selected_indices




def compose_music(music_model, cue=None, source_loader = None, topn=6, print_gen=False, encoder_decoder=False):  #cue-shape : (cue_len, 5)
    

    if type(cue) != type(None):
        cue = np.concatenate([np.zeros(cue.shape[:-1] + (1,)), cue], axis=-1)
        cue[np.any(cue, axis=-1)==False, 0] = 1 
        # Argmax results, one pitch at a time step for an instrument, ignores chords
        cue = np.expand_dims(cue.argmax(axis=-1), axis=0) #(800, 5)
    else:
        until = randint(0, 100)
        for _ in range(until):
            batch = next(source_loader)
        
        if encoder_decoder:
            x = batch[0][0]
        else:
            x = batch[0]
        
        cue = np.expand_dims(x[0], axis=0)


    composition = [cue[:, -1]]      #List[(1, 5,)]
    gen = 1

    while True:
 
        if print_gen:
            print("Generation : ", gen)
        gen += 1
        pcomp = np.expand_dims(np.concatenate(composition),axis=0)

        if encoder_decoder:
            pred = np.concatenate(music_model( [cue, pcomp] ))  #(5, 1, 129)
        else:
            pred = np.concatenate(music_model(pcomp))  #(5, 1, 129)

        if INSTRUMENTS == 1:
            pred = np.expand_dims(pred, axis=0)


  
        
        preds = []
        for instrument in range(INSTRUMENTS):
            probs = pred[instrument, -1]
            #selected_indices = top_p_sampling(probs, top_p)
            #new_probs = np.zeros(probs.shape)
            #new_probs[selected_indices] = probs[selected_indices]
        
            exclude_pred = np.argsort(probs)[:-topn]
            #print(np.sort(probs))
            probs[exclude_pred] = 0.
            #print(probs)
            new_probs = probs
            new_probs = new_probs/np.sum(new_probs)
            
            preds += [np.random.choice(129, (1,), p=new_probs)]

        preds = np.array(preds)
        currcomp = preds.T #(1, 5)
        composition += [currcomp]
    
        yield np.concatenate(composition)
    

def get_avg_tempo(dir='lpd_5/lpd_5_full/0', topn=1000):
    samples = os.listdir(dir)[:topn]
    tempo = 0.
    count = 0
    for sample in samples:
        with np.load(os.path.join(dir, sample)) as data:
            tempo += np.sum(data['tempo'])
            count += data['tempo'].shape[0]
    return tempo/count

import numpy as np




def make_track(composition, tempo=120, from_model=True):


    tracks = []
    tempo = np.full(composition.shape[0], tempo)  #get the tempo

    
    track_data = {0 : ['Drums', 0], 1: ['Piano', 0], 2: ['Guitar', 24], 3:['Bass', 32], 4:['Strings', 48]} #{"is_drum": false, "program": 0, "name": "Piano"}, "0": {"is_drum": true, "program": 0, "name": "Drums"}, "3": {"is_drum": false, "program": 32, "name": "Bass"}, "2": {"is_drum": false, "program": 24, "name": "Guitar"}, "4": {"is_drum": false, "program": 48, "name": "Strings"}, "beat_resolution": 24}'
    if INSTRUMENTS == 1:
        track_data = {0:track_data[1]}
        
    track_names = [t[0] for t in track_data.values()]

    # Create a Track object for each track in the multitrack representation
    for i, track_name_program in track_data.items():
        
        track_name, program = track_name_program

        if from_model:
            piano_roll = CategoryEncoding(129, output_mode='one_hot')(composition[:, i]).numpy()[:, 1:]
        else:
            piano_roll = composition[:, i]
        
        # Create a Track object without providing the piano_roll argument
        track = ppr.BinaryTrack(name=track_name)
        
        # Assign piano roll data to the Track object
        track.pianoroll = piano_roll  # Assuming piano_roll is a single-track piano roll
        track.program = program  # Specify the program number if necessary
        
        if track_name == 'Drums':
            track.is_drum = True
        # Append the Track object to the list
        tracks.append(track)

    # Create a Multitrack object and assign the tracks to it
    multitrack = ppr.Multitrack(tracks=tracks, tempo=tempo, resolution=8)

    return multitrack


from collections import Counter as C
from copy import deepcopy
def get_class_weights(source_loader, steps=200, encoder_decoder=True):


    default = {k:0 for k in range(PITCHES)}
    class_weights = [default for _ in range(INSTRUMENTS)]
    for _ in range(steps):
        x, _ = next(source_loader)  #(batch_size, time_steps, 5)
        
        if encoder_decoder:
            x = x[0]

       
        for i in range(INSTRUMENTS):
            pcwd = class_weights[i]
            ncwd = C(x[:, :, i].ravel().tolist())
            for k in pcwd.keys():
                pcwd[k] += ncwd[k]

            class_weights[i] = pcwd

            #class_weights[i][0] += np.sum(1-(x[:, :, i]))
            #class_weights[i][1] += np.sum(x[:, :, i])
    


    for j in range(INSTRUMENTS):
        cwd = class_weights[j]
        total = sum(cwd.values())
        keys = deepcopy(list(cwd.keys()))
        for k in keys: 
            cwd[k] = 1 - (cwd[k])/total

        class_weights[j] = cwd
    
    if INSTRUMENTS == 1:
        return class_weights[0]
    else:
        return class_weights







In [44]:
def recurrent_encoder_decoder(pitches=PITCHES, instruments=INSTRUMENTS):
    Xinp = Input((None, instruments))
    Xpromptinp = Input((None, instruments))
    note_to_vec = Sequential([Embedding(PITCHES, 30), 
                              MaxPooling1D(INSTRUMENTS),
                              Flatten()
    ])

    X = TimeDistributed(note_to_vec)(Xinp)
    X = Bidirectional(LSTM(100, return_sequences=True))(X)
    X = Bidirectional(LSTM(100, return_sequences=True))(X)
    X = Bidirectional(LSTM(100, return_sequences=True))(X)
    X = TimeDistributed(Dropout(0.2))(X)
    _, *internal_state = LSTM(100, return_state=True)(X)

    Y = TimeDistributed(note_to_vec)(Xpromptinp)
    Y = LSTM(100, return_sequences=True)(Y, initial_state=internal_state)
    Y = TimeDistributed(Dense(100, 'relu'))(Y)
    Y = TimeDistributed(Dropout(0.3))(Y)
    Out = []
    for instrument in range(instruments):
        Out += [TimeDistributed(Dense(pitches, 'softmax'), name=f'instrument_{instrument+1}')(Y)]
    

    losses = ['sparse_categorical_crossentropy']*instruments
    if instruments == 1:
        Out = Out[0]
        losses = losses[0]
    
    In = [Xinp, Xpromptinp]
    model = Model(In, Out)
    model.compile(Adam(1e-3), loss=losses, metrics=['accuracy'])
    return model

def recurrent(pitches=PITCHES, instruments=INSTRUMENTS):
    Xinp = Input((None, instruments))
    X = LSTM(200, return_sequences=True)(Xinp)
    X = LSTM(100, return_sequences=True)(X)
    X = TimeDistributed(Dense(50, 'relu'))(X)
    
    Out = []
    for instrument in range(instruments):
        Out += [TimeDistributed(Dense(pitches, 'softmax'), name=f'instrument_{instrument+1}')(X)]
    
    losses = ['sparse_categorical_crossentropy']*instruments
    if instruments == 1:
        Out = Out[0]
        losses = losses[0]
    
    In = Xinp 
    model = Model(In, Out)
    model.compile(Adam(1e-3), loss=losses, metrics=['accuracy'])
    return model



model = recurrent_encoder_decoder()
model.summary()




ERROR:tensorflow:==================================
Object was never used (type <class 'tensorflow.python.ops.tensor_array_ops.TensorArray'>):
If you want to mark it as used call its "mark_used()" method.
It was originally created here:
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/backend.py", line 5158, in <genexpr>
    output_ta_t = tuple(  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tensorflow/python/util/tf_should_use.py", line 288, in wrapped
    return _add_should_use_warning(fn(*args, **kwargs),


ERROR:tensorflow:==================================
Object was never used (type <class 'tensorflow.python.ops.tensor_array_ops.TensorArray'>):
If you want to mark it as used call its "mark_used()" method.
It was originally created here:
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/backend.py", line 5158, in <genexpr>
    output_ta_t = tuple(  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tensorflow/python/util/tf_should_use.py", line 288, in wrapped
    return _add_should_use_warning(fn(*args, **kwargs),


ERROR:tensorflow:==================================
Object was never used (type <class 'tensorflow.python.ops.tensor_array_ops.TensorArray'>):
If you want to mark it as used call its "mark_used()" method.
It was originally created here:
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/backend.py", line 5158, in <genexpr>
    output_ta_t = tuple(  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tensorflow/python/util/tf_should_use.py", line 288, in wrapped
    return _add_should_use_warning(fn(*args, **kwargs),


ERROR:tensorflow:==================================
Object was never used (type <class 'tensorflow.python.ops.tensor_array_ops.TensorArray'>):
If you want to mark it as used call its "mark_used()" method.
It was originally created here:
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/backend.py", line 5158, in <genexpr>
    output_ta_t = tuple(  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tensorflow/python/util/tf_should_use.py", line 288, in wrapped
    return _add_should_use_warning(fn(*args, **kwargs),


ERROR:tensorflow:==================================
Object was never used (type <class 'tensorflow.python.ops.tensor_array_ops.TensorArray'>):
If you want to mark it as used call its "mark_used()" method.
It was originally created here:
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/backend.py", line 5158, in <genexpr>
    output_ta_t = tuple(  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tensorflow/python/util/tf_should_use.py", line 288, in wrapped
    return _add_should_use_warning(fn(*args, **kwargs),


ERROR:tensorflow:==================================
Object was never used (type <class 'tensorflow.python.ops.tensor_array_ops.TensorArray'>):
If you want to mark it as used call its "mark_used()" method.
It was originally created here:
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/backend.py", line 5158, in <genexpr>
    output_ta_t = tuple(  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tensorflow/python/util/tf_should_use.py", line 288, in wrapped
    return _add_should_use_warning(fn(*args, **kwargs),


ERROR:tensorflow:==================================
Object was never used (type <class 'tensorflow.python.ops.tensor_array_ops.TensorArray'>):
If you want to mark it as used call its "mark_used()" method.
It was originally created here:
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/backend.py", line 5158, in <genexpr>
    output_ta_t = tuple(  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tensorflow/python/util/tf_should_use.py", line 288, in wrapped
    return _add_should_use_warning(fn(*args, **kwargs),


ERROR:tensorflow:==================================
Object was never used (type <class 'tensorflow.python.ops.tensor_array_ops.TensorArray'>):
If you want to mark it as used call its "mark_used()" method.
It was originally created here:
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/backend.py", line 5158, in <genexpr>
    output_ta_t = tuple(  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tensorflow/python/util/tf_should_use.py", line 288, in wrapped
    return _add_should_use_warning(fn(*args, **kwargs),


Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_3 (InputLayer)        [(None, None, 1)]            0         []                            
                                                                                                  
 time_distributed_5 (TimeDi  (None, None, 30)             3870      ['input_3[0][0]']             
 stributed)                                                                                       
                                                                                                  
 bidirectional_1 (Bidirecti  (None, None, 200)            104800    ['time_distributed_5[0][0]']  
 onal)                                                                                            
                                                                                            

In [245]:
#output_sequence_len = 0 if not encoder_decoder else >=1

# Data INFO
# ~175K midi files
# Files could be short melodies, loop compositions or long symphonies or songs
# ~10K midi files in each of the 16 top level subdirectories 
# Considering 300 midi files at present, at 8 resolution..~ 0.2% Dataset
# Generating a polyphonic monophony music (multiple instruments each following a monophonic melody line independently) or type of contrapuntal 
train_source, val_source, test_source  = process_dataset_in_chunks('./lpd_5/lpd_5_full/0', input_sequence_len=8*100, output_sequence_len=8*400, batch_size=128, topn=600, train_size=0.7, val_size=0.2, resolution=8, prune_rest_note_percent=0.7, encoder_decoder=True)

Preparing Test Dataset...: 100%|██████████| 180/180 [00:02<00:00, 75.50it/s]


In [45]:
train_source = ('lpd_5_batched/train_inputs', 'lpd_5_batched/train_outputs')
music_loader = load_music_batches(*train_source, encoder_decoder=True)
steps = len(os.listdir(train_source[0]))-1
steps

243

In [46]:
val_source = ('lpd_5_batched/val_inputs', 'lpd_5_batched/val_outputs')
val_music_loader = load_music_batches(*val_source, encoder_decoder=True)
val_steps = len(os.listdir(val_source[0]))-1
val_steps

243

In [47]:
history = model.fit(music_loader, steps_per_epoch=steps, validation_data=val_music_loader, validation_steps=val_steps, epochs=3)

Epoch 1/3
243/243 [==============================] - 6371s 26s/step - loss: 3.5902 - accuracy: 0.1425 - val_loss: 2.9075 - val_accuracy: 0.2145
Epoch 2/3
243/243 [==============================] - 2263s 9s/step - loss: 2.1526 - accuracy: 0.4861 - val_loss: 1.1274 - val_accuracy: 0.7890
Epoch 3/3
243/243 [==============================] - 2327s 10s/step - loss: 1.5081 - accuracy: 0.6953 - val_loss: 0.7699 - val_accuracy: 0.8984


In [271]:
model.save('encoder-decoder-bilstm-600topn-ep5.5_1instrument')

ERROR:tensorflow:==================================
Object was never used (type <class 'tensorflow.python.ops.tensor_array_ops.TensorArray'>):
If you want to mark it as used call its "mark_used()" method.
It was originally created here:
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/backend.py", line 5158, in <genexpr>
    output_ta_t = tuple(  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tensorflow/python/util/tf_should_use.py", line 288, in wrapped
    return _add_should_use_warning(fn(*args, **kwargs),


ERROR:tensorflow:==================================
Object was never used (type <class 'tensorflow.python.ops.tensor_array_ops.TensorArray'>):
If you want to mark it as used call its "mark_used()" method.
It was originally created here:
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/backend.py", line 5158, in <genexpr>
    output_ta_t = tuple(  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tensorflow/python/util/tf_should_use.py", line 288, in wrapped
    return _add_should_use_warning(fn(*args, **kwargs),


ERROR:tensorflow:==================================
Object was never used (type <class 'tensorflow.python.ops.tensor_array_ops.TensorArray'>):
If you want to mark it as used call its "mark_used()" method.
It was originally created here:
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/backend.py", line 5158, in <genexpr>
    output_ta_t = tuple(  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tensorflow/python/util/tf_should_use.py", line 288, in wrapped
    return _add_should_use_warning(fn(*args, **kwargs),


ERROR:tensorflow:==================================
Object was never used (type <class 'tensorflow.python.ops.tensor_array_ops.TensorArray'>):
If you want to mark it as used call its "mark_used()" method.
It was originally created here:
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/backend.py", line 5158, in <genexpr>
    output_ta_t = tuple(  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tensorflow/python/util/tf_should_use.py", line 288, in wrapped
    return _add_should_use_warning(fn(*args, **kwargs),


ERROR:tensorflow:==================================
Object was never used (type <class 'tensorflow.python.ops.tensor_array_ops.TensorArray'>):
If you want to mark it as used call its "mark_used()" method.
It was originally created here:
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/backend.py", line 5158, in <genexpr>
    output_ta_t = tuple(  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tensorflow/python/util/tf_should_use.py", line 288, in wrapped
    return _add_should_use_warning(fn(*args, **kwargs),


ERROR:tensorflow:==================================
Object was never used (type <class 'tensorflow.python.ops.tensor_array_ops.TensorArray'>):
If you want to mark it as used call its "mark_used()" method.
It was originally created here:
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/backend.py", line 5158, in <genexpr>
    output_ta_t = tuple(  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tensorflow/python/util/tf_should_use.py", line 288, in wrapped
    return _add_should_use_warning(fn(*args, **kwargs),


INFO:tensorflow:Assets written to: encoder-decoder-bilstm-600topn-ep5.5_1instrument/assets


INFO:tensorflow:Assets written to: encoder-decoder-bilstm-600topn-ep5.5_1instrument/assets


In [5]:
model = tf.keras.saving.load_model('encoder-decoder-bilstm-600topn-ep5.5_1instrument')

In [8]:
one_batch = next(music_loader)

In [48]:
test_source = ('lpd_5_batched/test_inputs', 'lpd_5_batched/test_outputs')
test_input_dir, test_output_dir = test_source
test_inputs = os.listdir(test_input_dir)
test_outputs = os.listdir(test_output_dir)

sample_index = randint(0, len(test_inputs)-1)
x = np.load(os.path.join(test_input_dir, test_inputs[sample_index]))
y = np.load(os.path.join(test_output_dir, test_outputs[sample_index]))

x.shape, y.shape

((800, 1, 128), (4901, 1, 128))

In [49]:
print(x.argmax(-1))

[[ 0]
 [ 0]
 [ 0]
 [36]
 [36]
 [36]
 [36]
 [36]
 [36]
 [36]
 [36]
 [36]
 [36]
 [36]
 [36]
 [36]
 [36]
 [48]
 [48]
 [48]
 [48]
 [48]
 [48]
 [48]
 [48]
 [48]
 [48]
 [48]
 [48]
 [48]
 [48]
 [48]
 [48]
 [48]
 [48]
 [48]
 [48]
 [48]
 [48]
 [60]
 [62]
 [62]
 [62]
 [62]
 [62]
 [62]
 [63]
 [63]
 [63]
 [46]
 [46]
 [46]
 [46]
 [46]
 [46]
 [46]
 [46]
 [46]
 [46]
 [46]
 [46]
 [46]
 [46]
 [46]
 [46]
 [46]
 [46]
 [46]
 [46]
 [46]
 [46]
 [46]
 [46]
 [46]
 [46]
 [46]
 [46]
 [46]
 [46]
 [46]
 [46]
 [46]
 [46]
 [46]
 [46]
 [60]
 [60]
 [60]
 [60]
 [62]
 [62]
 [62]
 [63]
 [63]
 [63]
 [41]
 [41]
 [41]
 [41]
 [41]
 [41]
 [41]
 [41]
 [41]
 [48]
 [48]
 [48]
 [48]
 [53]
 [53]
 [53]
 [53]
 [53]
 [56]
 [56]
 [56]
 [56]
 [56]
 [56]
 [56]
 [56]
 [56]
 [56]
 [56]
 [56]
 [56]
 [56]
 [56]
 [56]
 [56]
 [56]
 [56]
 [56]
 [56]
 [72]
 [72]
 [72]
 [72]
 [72]
 [72]
 [72]
 [44]
 [44]
 [44]
 [44]
 [44]
 [44]
 [44]
 [44]
 [44]
 [44]
 [44]
 [51]
 [51]
 [51]
 [51]
 [51]
 [51]
 [51]
 [51]
 [51]
 [51]
 [51]
 [51]
 [51]
 [51]
 [51

In [54]:
composer = compose_music(music_model=model, source_loader=music_loader, topn=5, encoder_decoder=True, print_gen=True)



In [55]:
total_track_length = y.shape[0]
composition_length = 200 #int(0.1*total_track_length)
TEMPO = 60

print(f"Composing for {composition_length} time steps at {TEMPO} bpm")
for _ in range(composition_length):
    composition = next(composer)
    print(composition[-1])
  


output_midi_path = f'generated_track.mid'
output_audio_path = f'generated_track.wav'

print("Composition Done...")
generated_track = make_track(composition, tempo=TEMPO)
print("Track Done...")

multitrack_to_midi(generated_track, output_midi_path)
print("Generated Midi file Done...")
midi_to_wav(output_midi_path, output_audio_path)
print("Generated Audio file Done...")



output_midi_path = 'original_track.mid'
output_audio_path = 'original_track.wav'

original_track = make_track(composition=y[:composition_length], tempo=TEMPO, from_model=False)
print("Original Track Done...")

multitrack_to_midi(original_track, output_midi_path)
print("Original Midi file Done...")
midi_to_wav(output_midi_path, output_audio_path)
print("Original Audio file Done...")

Composing for 200 time steps at 60 bpm
Generation :  1
[65]
Generation :  2
[65]
Generation :  3
[65]
Generation :  4
[63]
Generation :  5
[63]
Generation :  6
[63]
Generation :  7
[63]
Generation :  8
[63]
Generation :  9
[62]
Generation :  10
[63]
Generation :  11
[60]
Generation :  12
[60]
Generation :  13
[60]
Generation :  14
[60]
Generation :  15
[63]
Generation :  16
ERROR:tensorflow:==================================
Object was never used (type <class 'tensorflow.python.ops.tensor_array_ops.TensorArray'>):
If you want to mark it as used call its "mark_used()" method.
It was originally created here:
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/backend.py", line 5158, in <genexpr>
    output_ta_t = tuple(  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tensorflow/python/util/tf_should_use.py", line 288, in wrapped
    return _add_should_use_warning(fn(*args, **kwargs),


ERROR:tensorflow:==================================
Object was never used (type <class 'tensorflow.python.ops.tensor_array_ops.TensorArray'>):
If you want to mark it as used call its "mark_used()" method.
It was originally created here:
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/backend.py", line 5158, in <genexpr>
    output_ta_t = tuple(  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tensorflow/python/util/tf_should_use.py", line 288, in wrapped
    return _add_should_use_warning(fn(*args, **kwargs),


[63]
Generation :  17
[60]
Generation :  18
[60]
Generation :  19
[60]
Generation :  20
[60]
Generation :  21
[60]
Generation :  22
[60]
Generation :  23
[55]
Generation :  24
[60]
Generation :  25
[60]
Generation :  26
[60]
Generation :  27
[60]
Generation :  28
[60]
Generation :  29
[56]
Generation :  30
[60]
Generation :  31
[60]
Generation :  32
[60]
Generation :  33
[60]
Generation :  34
[55]
Generation :  35
[55]
Generation :  36
[55]
Generation :  37
[55]
Generation :  38
[55]
Generation :  39
[55]
Generation :  40
[60]
Generation :  41
[60]
Generation :  42
[60]
Generation :  43
[60]
Generation :  44
[60]
Generation :  45
[60]
Generation :  46
[60]
Generation :  47
[55]
Generation :  48
[55]
Generation :  49
[55]
Generation :  50
[55]
Generation :  51
[55]
Generation :  52
[55]
Generation :  53
[55]
Generation :  54
[60]
Generation :  55
[60]
Generation :  56
[60]
Generation :  57
[60]
Generation :  58
[60]
Generation :  59
[60]
Generation :  60
[51]
Generation :  61
[51]
Gener

In [263]:
y.argmax(-1)[:100]

array([[58],
       [58],
       [58],
       [58],
       [58],
       [58],
       [58],
       [58],
       [58],
       [58],
       [58],
       [58],
       [58],
       [58],
       [58],
       [58],
       [58],
       [58],
       [58],
       [58],
       [58],
       [58],
       [58],
       [58],
       [58],
       [58],
       [58],
       [58],
       [58],
       [58],
       [74],
       [55],
       [55],
       [55],
       [55],
       [55],
       [55],
       [55],
       [55],
       [55],
       [55],
       [55],
       [55],
       [55],
       [55],
       [55],
       [55],
       [55],
       [55],
       [55],
       [55],
       [55],
       [55],
       [55],
       [55],
       [55],
       [55],
       [55],
       [55],
       [55],
       [55],
       [55],
       [55],
       [55],
       [55],
       [55],
       [55],
       [55],
       [58],
       [58],
       [58],
       [58],
       [62],
       [62],
       [62],
       [62],
       [67],

In [257]:
print(y.argmax(-1) == composition)

False


/var/folders/zk/wsnb2q8d20d9ps6y9yfkkg000000gn/T/ipykernel_1464/1996242684.py:1: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  print(y.argmax(-1) == composition)


In [258]:
composition[1:][:10], y[:277].argmax(-1)[:10]

(array([[55],
        [55],
        [55],
        [55],
        [55],
        [55],
        [48],
        [48],
        [48],
        [48]]),
 array([[54],
        [54],
        [54],
        [54],
        [54],
        [54],
        [54],
        [54],
        [54],
        [54]]))

In [194]:
ind = 0
pred_batch = model.predict([x[ind:ind+1] for x in one_batch[0]])
pred_composition = [pred_batch[0].argmax(-1) for inst in range(INSTRUMENTS)]
true_composition = [one_batch[1][ind].numpy() for inst in range(INSTRUMENTS)]
pred_composition

1/1 [==============================] - 0s 63ms/step


[array([37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 39,
        39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39,
        39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39,
        39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39,
        39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39,
        39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 44, 44, 44, 44, 44, 44,
        44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44,
        44, 44, 44, 44, 44, 44, 44, 44, 37, 37, 37, 37, 37, 37, 37, 37, 37,
        37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37,
        37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37,
        37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37,
        37, 37, 37, 37, 37, 46, 46, 46, 46, 46, 46, 46, 46])]

In [35]:
pred_batch.shape, one_batch[1].shape

((1, 600, 129), TensorShape([128, 600]))

In [195]:
true_composition

[array([37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 39, 39,
        39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39,
        39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39,
        39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39,
        39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39,
        39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 44, 44, 44, 44, 44, 44, 44,
        44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44,
        44, 44, 44, 44, 44, 44, 44, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37,
        37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37,
        37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37,
        37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37,
        37, 37, 37, 37, 46, 46, 46, 46, 46, 46, 46, 46, 46])]

In [346]:
model.evaluate(val_music_loader, steps=steps)

689/689 [==============================] - 643s 932ms/step - loss: 7.3065 - instrument_1_loss: 0.8514 - instrument_2_loss: 1.9456 - instrument_3_loss: 1.4665 - instrument_4_loss: 0.9309 - instrument_5_loss: 2.1122 - instrument_1_accuracy: 0.8239 - instrument_2_accuracy: 0.5091 - instrument_3_accuracy: 0.5503 - instrument_4_accuracy: 0.7740 - instrument_5_accuracy: 0.4575


[7.3065385818481445,
 0.8513951301574707,
 1.9455796480178833,
 1.466457724571228,
 0.9308863878250122,
 2.112215518951416,
 0.8239405155181885,
 0.5091147422790527,
 0.5502922534942627,
 0.7740278244018555,
 0.4575483500957489]